# 단순 선형회귀 분석 - 캘리포니아 주택가격 데이터셋

## 학습 목표
- 단순 선형회귀의 개념과 원리 이해
- 단변수, 다변수, 전체 특성을 사용한 선형회귀 모델 비교
- 모델 성능 평가 지표 (R², MSE) 이해
- 데이터 시각화를 통한 회귀 분석 결과 해석

## 데이터셋 소개
**캘리포니아 주택가격 데이터셋 (California Housing Dataset)**
- **목표**: 캘리포니아 지역의 주택가격 예측
- **특성 수**: 8개 (MedInc, HouseAge, AveRooms, AveBedrms, Population, AveOccup, Latitude, Longitude)
- **샘플 수**: 20,640개
- **목표 변수**: 주택가격 (단위: $100,000)

이 노트북에서는 다음과 같은 분석을 수행합니다:
1. **단변수 선형회귀**: MedInc(중간 소득) → 주택가격
2. **다변수 선형회귀**: MedInc + AveRooms → 주택가격  
3. **전체 특성 선형회귀**: 모든 8개 특성 → 주택가격

In [1]:
# 필요한 라이브러리 import
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split  # 개선된 데이터 분할을 위해 추가
from sklearn.preprocessing import MinMaxScaler  # 데이터 정규화를 위해 추가
import pandas as pd

# Windows에서 한글 폰트 설정 (필요시 주석 해제)
import matplotlib.font_manager as fm
font_list = [font.name for font in fm.fontManager.ttflist]
if 'Malgun Gothic' in font_list:
    plt.rcParams['font.family'] = 'Malgun Gothic'
elif 'NanumGothic' in font_list:
    plt.rcParams['font.family'] = 'NanumGothic'

plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지

# 시각화 설정
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)

## 1. 데이터 로드 및 탐색

In [2]:
# California Housing 데이터셋 로드
# 주택 가격 예측을 위한 데이터 (특성 간 상관관계가 다양함)
housing = datasets.fetch_california_housing()

print("특성 이름:")
print(housing.feature_names)
print(f"\n데이터 형태: {housing.data.shape}")
print(f"목표 변수 형태: {housing.target.shape}")
print(f"\n특성 설명:")
for i, name in enumerate(housing.feature_names):
    print(f"  {i}: {name}")

특성 이름:
['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']

데이터 형태: (20640, 8)
목표 변수 형태: (20640,)

특성 설명:
  0: MedInc
  1: HouseAge
  2: AveRooms
  3: AveBedrms
  4: Population
  5: AveOccup
  6: Latitude
  7: Longitude


### 데이터셋 기본 정보

In [3]:
# DataFrame으로 변환하여 데이터 탐색
df_housing = pd.DataFrame(data=housing.data, columns=housing.feature_names)
df_housing['target'] = housing.target  # 목표 변수도 추가

print("DataFrame 형태:")
print(df_housing.head())
print(f"\nDataFrame 정보:")
print(df_housing.info())

DataFrame 형태:
   MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1  8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86   
2  7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85   
3  5.6431      52.0  5.817352   1.073059       558.0  2.547945     37.85   
4  3.8462      52.0  6.281853   1.081081       565.0  2.181467     37.85   

   Longitude  target  
0    -122.23   4.526  
1    -122.22   3.585  
2    -122.24   3.521  
3    -122.25   3.413  
4    -122.25   3.422  

DataFrame 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   MedInc      20640 non-null  float64
 1   HouseAge    20640 non-null  float64
 2   AveRooms    20640 non-null  float64
 3   AveBedrms   20640 non-null  float64
 4   Population  20640

### 데이터셋 상세 정보

In [4]:
# MedInc(중간 소득) 특성만 사용하여 단변수 선형회귀 데이터 준비
# reshape(-1, 1): 1차원 배열을 2차원으로 변환 (sklearn 요구사항)
X_medinc = df_housing['MedInc'].values.reshape(-1, 1)
y = housing.target

print(f"MedInc 데이터 형태: {X_medinc.shape}")
print(f"목표 변수 형태: {y.shape}")
print(f"MedInc 범위: ${X_medinc.min():.2f} ~ ${X_medinc.max():.2f}")
print(f"MedInc 평균: ${X_medinc.mean():.2f}")

MedInc 데이터 형태: (20640, 1)
목표 변수 형태: (20640,)
MedInc 범위: $0.50 ~ $15.00
MedInc 평균: $3.87


### 데이터 분할

In [5]:
# 개선된 데이터 분할 방법 사용
# train_test_split을 사용하여 더 체계적으로 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X_medinc, y, test_size=0.2, random_state=42
)

# MinMaxScaler를 사용한 데이터 정규화
scaler_single = MinMaxScaler()
X_train_scaled = scaler_single.fit_transform(X_train)
X_test_scaled = scaler_single.transform(X_test)

print(f"훈련 데이터 크기: {X_train.shape}")
print(f"테스트 데이터 크기: {X_test.shape}")
print(f"정규화 전 MedInc 범위: {X_train.min():.2f} ~ {X_train.max():.2f}")
print(f"정규화 후 MedInc 범위: {X_train_scaled.min():.2f} ~ {X_train_scaled.max():.2f}")

훈련 데이터 크기: (16512, 1)
테스트 데이터 크기: (4128, 1)
정규화 전 MedInc 범위: 0.50 ~ 15.00
정규화 후 MedInc 범위: 0.00 ~ 1.00


In [6]:
# 선형회귀 모델 생성 및 훈련 (정규화된 데이터 사용)
regr = linear_model.LinearRegression()

# 정규화된 훈련 데이터로 모델 학습
regr.fit(X_train_scaled, y_train)

print("모델 훈련 완료!")
print(f"회귀 계수 (기울기): {regr.coef_[0]:.2f}")
print(f"절편: {regr.intercept_:.2f}")
print(f"\n회귀식: 주택가격 = {regr.coef_[0]:.2f} * MedInc(정규화) + {regr.intercept_:.2f}")
print(f"해석: 정규화된 중간 소득이 1단위 증가하면 주택가격이 {regr.coef_[0]:.2f}만 달러 증가")

모델 훈련 완료!
회귀 계수 (기울기): 6.08
절편: 0.65

회귀식: 주택가격 = 6.08 * MedInc(정규화) + 0.65
해석: 정규화된 중간 소득이 1단위 증가하면 주택가격이 6.08만 달러 증가


In [7]:
# 모델 예측 및 성능 평가 (정규화된 테스트 데이터 사용)
y_pred = regr.predict(X_test_scaled)

# 모델 성능 평가
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("="*50)
print("단변수 선형회귀 모델 성능 평가 (정규화 적용)")
print("="*50)
print(f"평균 제곱 오차 (MSE): {mse:.2f}")
print(f"결정계수 (R²): {r2:.2f}")
print(f"\n해석:")
print(f"- R² = {r2:.2f}는 MedInc가 주택가격의 {r2*100:.0f}%를 설명함")
print(f"- 나머지 {(1-r2)*100:.0f}%는 다른 요인들에 의해 결정됨")
print(f"- MSE = {mse:.2f}는 예측값과 실제값의 평균 제곱 차이")
print(f"- RMSE = {np.sqrt(mse):.2f} (평균 예측 오차)")

단변수 선형회귀 모델 성능 평가 (정규화 적용)
평균 제곱 오차 (MSE): 0.71
결정계수 (R²): 0.46

해석:
- R² = 0.46는 MedInc가 주택가격의 46%를 설명함
- 나머지 54%는 다른 요인들에 의해 결정됨
- MSE = 0.71는 예측값과 실제값의 평균 제곱 차이
- RMSE = 0.84 (평균 예측 오차)


## 전체 특성 선형회귀 (모든 특성 → 주택가격)

### 전체 특성 데이터 분할

In [8]:
# 전체 특성을 사용한 선형회귀 데이터 준비
X_all = df_housing.drop('target', axis=1).values  # target 컬럼 제외한 모든 특성
y = housing.target

print(f"전체 특성 데이터 형태: {X_all.shape}")
print(f"사용된 특성: {list(df_housing.columns[:-1])}")  # target 제외
print(f"특성 수: {X_all.shape[1]}개")

전체 특성 데이터 형태: (20640, 8)
사용된 특성: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
특성 수: 8개


In [9]:
# 전체 특성 선형회귀를 위한 데이터 분할
X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(
    X_all, y, test_size=0.2, random_state=42
)

# MinMaxScaler를 사용한 전체 특성 데이터 정규화
scaler_all = MinMaxScaler()
X_train_all_scaled = scaler_all.fit_transform(X_train_all)
X_test_all_scaled = scaler_all.transform(X_test_all)

print(f"훈련 데이터 크기: {X_train_all.shape}, {y_train_all.shape}")
print(f"테스트 데이터 크기: {X_test_all.shape}, {y_test_all.shape}")
print(f"정규화 전 범위: {X_train_all.min():.2f} ~ {X_train_all.max():.2f}")
print(f"정규화 후 범위: {X_train_all_scaled.min():.2f} ~ {X_train_all_scaled.max():.2f}")

훈련 데이터 크기: (16512, 8), (16512,)
테스트 데이터 크기: (4128, 8), (4128,)
정규화 전 범위: -124.35 ~ 35682.00
정규화 후 범위: 0.00 ~ 1.00


In [10]:
# 전체 특성 선형회귀 모델 훈련 (정규화된 데이터 사용)
regr_all = linear_model.LinearRegression()
regr_all.fit(X_train_all_scaled, y_train_all)

print("전체 특성 선형회귀 모델 훈련 완료!")
print(f"회귀계수: {regr_all.coef_}")
print(f"절편: {regr_all.intercept_:.2f}")
print(f"사용된 특성 수: {len(regr_all.coef_)}개 (정규화 적용)")
print(f"특성명: {list(df_housing.columns[:-1])}")  # target 제외

전체 특성 선형회귀 모델 훈련 완료!
회귀계수: [  6.50587593   0.49593713 -17.39108272  19.81593931  -0.07241483
  -4.38194802  -3.94604937  -4.35442897]
절편: 3.63
사용된 특성 수: 8개 (정규화 적용)
특성명: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']


In [12]:
# 전체 특성 모델 예측 및 성능 평가 (정규화된 테스트 데이터 사용)
y_pred_all = regr_all.predict(X_test_all_scaled)

mse_all = mean_squared_error(y_test_all, y_pred_all)
r2_all = r2_score(y_test_all, y_pred_all)

print("="*50)
print("전체 특성 선형회귀 모델 성능 평가 (정규화 적용)")
print("="*50)
print(f"평균 제곱 오차 (MSE): {mse_all:.2f}")
print(f"결정계수 (R²): {r2_all:.2f}")
print(f"\n해석:")
print(f"- R² = {r2_all:.2f}는 모든 특성이 주택가격의 {r2_all*100:.0f}%를 설명함")
print(f"- 단변수 모델 대비 성능 향상: R² {r2:.2f} → {r2_all:.2f} ({r2_all-r2:+.2f})")

전체 특성 선형회귀 모델 성능 평가 (정규화 적용)
평균 제곱 오차 (MSE): 0.56
결정계수 (R²): 0.58

해석:
- R² = 0.58는 모든 특성이 주택가격의 58%를 설명함
- 단변수 모델 대비 성능 향상: R² 0.46 → 0.58 (+0.12)


## 5. 모델 성능 비교 및 결과 해석

In [17]:
print("="*60)
print("5. 학습 요약 및 결론")
print("="*60)

print("""
📊 **실험 결과 요약**

1. **단변수 선형회귀 (MedInc만 사용)**
   - R² = {:.2f} (MedInc가 주택가격의 {:.0f}% 설명)
   - MSE = {:.2f}

2. **전체 특성 선형회귀 (8개 특성)**
   - R² = {:.2f} (모든 특성이 주택가격의 {:.0f}% 설명)
   - MSE = {:.2f}
   - 단변수 대비 성능 향상: {:.2f}

🔍 **주요 인사이트**
- California Housing 데이터셋에서는 특성 수에 따른 성능 차이가 뚜렷함
- MedInc(중간 소득)가 가장 중요한 예측 변수
- 모든 특성 사용 시 가장 높은 성능 달성
- 선형회귀는 해석이 용이하고 구현이 간단한 모델임
""".format(r2, r2*100, mse, r2_all, r2_all*100, mse_all, r2_all-r2))

5. 학습 요약 및 결론

📊 **실험 결과 요약**

1. **단변수 선형회귀 (MedInc만 사용)**
   - R² = 0.46 (MedInc가 주택가격의 46% 설명)
   - MSE = 0.71

2. **전체 특성 선형회귀 (8개 특성)**
   - R² = 0.58 (모든 특성이 주택가격의 58% 설명)
   - MSE = 0.56
   - 단변수 대비 성능 향상: 0.12

🔍 **주요 인사이트**
- California Housing 데이터셋에서는 특성 수에 따른 성능 차이가 뚜렷함
- MedInc(중간 소득)가 가장 중요한 예측 변수
- 모든 특성 사용 시 가장 높은 성능 달성
- 선형회귀는 해석이 용이하고 구현이 간단한 모델임

